In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
train_df = pd.read_csv("../input/short_prac_train.csv")
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test_df = pd.read_csv("../input/short_prac_test.csv")
test_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_X = train_df.iloc[:, 1:785]/255
train_Y = train_df[["label"]]
test_X = test_df.iloc[:, 1:785]/255
test_Y = test_df[["label"]]

In [5]:
x = tf.placeholder(tf.float32, [None, 784])

W1 = tf.Variable(tf.random_normal([784,512], mean=0.0, stddev=0.5))
b1 = tf.Variable(tf.random_normal([512], mean=0.0, stddev=0.1))
y1 = tf.matmul(x, W1) + b1
y1 = tf.nn.relu(y1)

W2 = tf.Variable(tf.random_normal([512,256], mean=0.0, stddev=0.5))
b2 = tf.Variable(tf.random_normal([256], mean=0.0, stddev=0.1))
y2 = tf.matmul(y1, W2) + b2
y2 = tf.nn.relu(y2)

W3 = tf.Variable(tf.random_normal([256,10], mean=0.0, stddev=0.5))
b3 = tf.Variable(tf.random_normal([10], mean=0.0, stddev=0.1))
y3 = tf.matmul(y2, W3) + b3

y = tf.placeholder(tf.int64, [None, 1])
y_ = tf.one_hot(indices = y, depth = 10)

In [6]:
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.01
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           10000, 1 - 1e-6, staircase=True)

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y3))
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum = 0.9, use_nesterov=True).minimize(cost, global_step = global_step)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [11]:
num_epochs = 10
num_data = train_X.shape[0]
batch_size = 100
for epoch in range(num_epochs):
    s_idx = np.random.permutation(num_data)
    for idx in range(0, num_data, batch_size):
        batch_x = train_X.iloc[s_idx[idx: idx + batch_size].tolist(),:]
        batch_y = train_Y.iloc[s_idx[idx: idx + batch_size].tolist()]
        sess.run(optimizer, feed_dict={x:batch_x, y:batch_y})

In [12]:
pred = sess.run(tf.nn.softmax(y3), feed_dict = {x: test_X})
pred_d = tf.argmax(pred,1)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(confusion_matrix(test_Y, sess.run(pred_d)), accuracy_score(test_Y, sess.run(pred_d)))

[[91  0  1  0  0  1  2  2  1  2]
 [ 0 96  0  0  0  0  1  3  0  0]
 [ 1  0 87  4  1  0  0  3  3  1]
 [ 0  2  5 77  0  9  0  1  4  2]
 [ 2  1  1  0 76  0  1  3  1 15]
 [ 2  1  2 11  0 76  1  1  5  1]
 [ 0  0  5  0  1  5 88  0  1  0]
 [ 1  2  0  0  4  0  0 87  1  5]
 [ 1  0  2  5  0  2  1  0 88  1]
 [ 0  0  1  2  8  0  0  4  2 83]] 0.849
